In [156]:
import sqlite3
import pandas as pd
import numpy as np
import pickle
import json
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [157]:
# 데이터베이스 연결
conn = sqlite3.connect('C:/Users/yunyo/project/movie_recommender/movies.db')

# 쿼리로 데이터 가져오기
query1 = "SELECT * FROM movies"
query2 = "SELECT * FROM movie_details"
df1 = pd.read_sql_query(query1, conn)
df2 = pd.read_sql_query(query2, conn)

# 데이터베이스 연결 해제
conn.close()

In [158]:
df1.head(3)

,movie_id,title,release_date,popularity,vote_average,vote_count,overview,poster_path,genres
0,5,Four Rooms,1995-12-09,18.413,5.8,2380,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name..."
1,6,Judgment Night,1993-10-15,9.805,6.6,285,"While racing to a boxing match, Frank, Mike, J...",/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam..."
2,11,Star Wars,1977-05-25,88.660,8.2,18755,Princess Leia is captured and held hostage by ...,/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""..."


In [159]:
df2.head(3)

,movie_id,cast,crew,keywords
0,5,"[{""adult"": false, ""gender"": 2, ""id"": 3129, ""kn...","[{""adult"": false, ""gender"": 1, ""id"": 3110, ""kn...","{""id"": 5, ""keywords"": [{""id"": 612, ""name"": ""ho..."
1,6,"[{""adult"": false, ""gender"": 2, ""id"": 2880, ""kn...","[{""adult"": false, ""gender"": 2, ""id"": 2042, ""kn...","{""id"": 6, ""keywords"": [{""id"": 520, ""name"": ""ch..."
2,11,"[{""adult"": false, ""gender"": 2, ""id"": 2, ""known...","[{""adult"": false, ""gender"": 2, ""id"": 1, ""known...","{""id"": 11, ""keywords"": [{""id"": 803, ""name"": ""a..."


In [160]:
df1.shape, df2.shape

((9959, 9), (9979, 4))

In [161]:
df1.columns

Index(['movie_id', 'title', 'release_date', 'popularity', 'vote_average',
       'vote_count', 'overview', 'poster_path', 'genres'],
      dtype='object')

In [162]:
df2.columns

Index(['movie_id', 'cast', 'crew', 'keywords'], dtype='object')

In [163]:
df = pd.merge(df1, df2, on='movie_id', how='inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9959 entries, 0 to 9958
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie_id      9959 non-null   int64  
 1   title         9959 non-null   object 
 2   release_date  9959 non-null   object 
 3   popularity    9959 non-null   float64
 4   vote_average  9959 non-null   float64
 5   vote_count    9959 non-null   int64  
 6   overview      9959 non-null   object 
 7   poster_path   9925 non-null   object 
 8   genres        9959 non-null   object 
 9   cast          9959 non-null   object 
 10  crew          9959 non-null   object 
 11  keywords      9959 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 933.8+ KB


In [164]:
df.head(5)

,movie_id,title,release_date,popularity,vote_average,vote_count,overview,poster_path,genres,cast,crew,keywords
0,5,Four Rooms,1995-12-09,18.413,5.8,2380,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name...","[{""adult"": false, ""gender"": 2, ""id"": 3129, ""kn...","[{""adult"": false, ""gender"": 1, ""id"": 3110, ""kn...","{""id"": 5, ""keywords"": [{""id"": 612, ""name"": ""ho..."
1,6,Judgment Night,1993-10-15,9.805,6.6,285,"While racing to a boxing match, Frank, Mike, J...",/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""adult"": false, ""gender"": 2, ""id"": 2880, ""kn...","[{""adult"": false, ""gender"": 2, ""id"": 2042, ""kn...","{""id"": 6, ""keywords"": [{""id"": 520, ""name"": ""ch..."
2,11,Star Wars,1977-05-25,88.660,8.2,18755,Princess Leia is captured and held hostage by ...,/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...","[{""adult"": false, ""gender"": 2, ""id"": 2, ""known...","[{""adult"": false, ""gender"": 2, ""id"": 1, ""known...","{""id"": 11, ""keywords"": [{""id"": 803, ""name"": ""a..."
3,12,Finding Nemo,2003-05-30,85.520,7.8,17741,"Nemo, an adventurous young clownfish, is unexp...",/eHuGQ10FUzK1mdOY69wF5pGgEf5.jpg,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","[{""adult"": false, ""gender"": 2, ""id"": 13, ""know...","[{""adult"": false, ""gender"": 2, ""id"": 7, ""known...","{""id"": 12, ""keywords"": [{""id"": 970, ""name"": ""p..."
4,13,Forrest Gump,1994-06-23,66.808,8.5,24789,A man with a low IQ has accomplished great thi...,/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""adult"": false, ""gender"": 2, ""id"": 31, ""know...","[{""adult"": false, ""gender"": 2, ""id"": 37, ""know...","{""id"": 13, ""keywords"": [{""id"": 422, ""name"": ""v..."


In [165]:
print(type(df.loc[0, 'genres']))
print(df.loc[0, 'genres'])

<class 'str'>
[{"id": 80, "name": "Crime"}, {"id": 35, "name": "Comedy"}]


In [166]:
# list로 변경
features = ['genres', 'cast', 'crew', 'keywords']
for feature in features:
    df[feature] = df[feature].apply(lambda x: json.loads(x))

In [167]:
print(type(df.loc[0, 'genres']))
print(type(df.loc[0, 'cast']))
print(type(df.loc[0, 'crew']))
print(type(df.loc[0, 'keywords']))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'dict'>


In [168]:
df.loc[0, 'keywords'].get('keywords')

[{'id': 612, 'name': 'hotel'},
 {'id': 613, 'name': "new year's eve"},
 {'id': 616, 'name': 'witch'},
 {'id': 622, 'name': 'bet'},
 {'id': 922, 'name': 'hotel room'},
 {'id': 2700, 'name': 'sperm'},
 {'id': 9706, 'name': 'anthology'},
 {'id': 12670, 'name': 'los angeles, california'},
 {'id': 160488, 'name': 'hoodlum'},
 {'id': 171366, 'name': 'multiple storylines'},
 {'id': 187056, 'name': 'woman director'}]

In [169]:
df['keywords_data'] = df['keywords'].apply(lambda x: x['keywords'])
df['keywords_data']

0       [{'id': 612, 'name': 'hotel'}, {'id': 613, 'na...
1       [{'id': 520, 'name': 'chicago, illinois'}, {'i...
2       [{'id': 803, 'name': 'android'}, {'id': 4270, ...
3       [{'id': 970, 'name': 'parent child relationshi...
4       [{'id': 422, 'name': 'vietnam veteran'}, {'id'...
                              ...                        
9954                                                   []
9955                                                   []
9956                                                   []
9957                                                   []
9958                                                   []
Name: keywords_data, Length: 9959, dtype: object

In [170]:
type(df.loc[0, 'keywords_data'])

list

In [171]:
df['release_date'].sort_values(ascending=False)
df = df[df['release_date'] <= '2023-06-18']

In [172]:
#   # literal_eval = str 타입 변경 -> 오류뜸
#   features = ['genres','cast', 'crew', 'keywords']
#   for feature in features:
#       df[feature] = df[feature].apply(literal_eval)

In [173]:
df[['title', 'vote_count', 'vote_average']].sort_values('vote_average', ascending=False)

,title,vote_count,vote_average
8369,Orgasm Lecture 2,1,10.0
9846,Waitress,1,10.0
9077,Pretty Young Sister 4,1,10.0
9340,Pretty Red Dress,1,10.0
9709,I Love Lizzy,1,10.0
...,...,...,...
9737,BS High,0,0.0
8171,Gentleman Spa,0,0.0
9162,Schoolmistress 4,0,0.0
9750,Bios: Os Paralamas do Sucesso,0,0.0


가중 평점 W  </br>
v : 해당 영화 총 투표수</br>
m : 평점을 부여하기 위한 최소 투표수</br>
R : 해당 영화 평균 평점</br>
C : 모든 영화 평균 평점</br></br>
W = (v/(v+m)*R) + (m/(m+v)*C)

In [174]:
# 전체 영화의 평균 평점
C = df['vote_average'].mean()
C

6.386712508848215

In [175]:
# 최소 평가 개수 : 60%
m = df['vote_count'].quantile(0.6)
m

808.0

In [176]:
qualified = df[df['vote_count'] >= m]
print(qualified.shape)
qualified['vote_count'].sort_values()

(3957, 13)


5584      808
3892      808
1065      809
6530      809
8831      809
        ...  
3718    28713
3534    29233
90      29870
5238    31605
3836    33836
Name: vote_count, Length: 3957, dtype: int64

In [177]:
# 가중 평점
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v / (v+m)*R) + (m/(m+v)*C)

In [178]:
qualified['score'] = qualified.apply(weighted_rating, axis=1)

C:\Users\yunyo\AppData\Local\Temp\ipykernel_13244\1579282308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['score'] = qualified.apply(weighted_rating, axis=1)


In [179]:
qualified[['title', 'score']].sort_values('score', ascending = False).head(100)

,title,score
171,The Shawshank Redemption,8.624504
149,The Godfather,8.601072
240,Schindler's List,8.480523
151,The Godfather Part II,8.447490
90,The Dark Knight,8.444340
...,...,...
179,Casablanca,7.936912
400,Harry Potter and the Prisoner of Azkaban,7.936142
7386,Klaus,7.934703
8170,Spider-Man: No Way Home,7.929044


In [180]:
qualified.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3957 entries, 0 to 9607
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       3957 non-null   int64  
 1   title          3957 non-null   object 
 2   release_date   3957 non-null   object 
 3   popularity     3957 non-null   float64
 4   vote_average   3957 non-null   float64
 5   vote_count     3957 non-null   int64  
 6   overview       3957 non-null   object 
 7   poster_path    3957 non-null   object 
 8   genres         3957 non-null   object 
 9   cast           3957 non-null   object 
 10  crew           3957 non-null   object 
 11  keywords       3957 non-null   object 
 12  keywords_data  3957 non-null   object 
 13  score          3957 non-null   float64
dtypes: float64(3), int64(2), object(9)
memory usage: 463.7+ KB


In [181]:
qualified.reset_index(drop=True, inplace=True)

In [182]:
# 컨텐츠 기반 필터링
qualified['overview']

0       It's Ted the Bellhop's first night on the job....
1       Princess Leia is captured and held hostage by ...
2       Nemo, an adventurous young clownfish, is unexp...
3       A man with a low IQ has accomplished great thi...
4       Lester Burnham, a depressed suburban father in...
                              ...                        
3952    Following the latest Ghostface killings, the f...
3953    When a lively young family moves in next door,...
3954    Discover the game-changing partnership between...
3955    For best friends Becky and Hunter, life is all...
3956    A new documentary that showcases the making of...
Name: overview, Length: 3957, dtype: object

In [183]:
qualified['overview'].isnull().sum()

0

In [184]:
# 백터화 - BOW
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(qualified['overview'])

print(tfidf_matrix.shape)

(3957, 17037)


In [185]:
tfidf_matrix

<3957x17037 sparse matrix of type '<class 'numpy.float64'>'
	with 94193 stored elements in Compressed Sparse Row format>

In [186]:
# 유사도 판단 - 코사인 유사도
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.00914395,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00914395, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [187]:
indices = pd.Series(qualified.index, index=qualified['title']).drop_duplicates()
indices

title
Four Rooms                            0
Star Wars                             1
Finding Nemo                          2
Forrest Gump                          3
American Beauty                       4
                                   ... 
Scream VI                          3952
A Man Called Otto                  3953
Air                                3954
Fall                               3955
Obi-Wan Kenobi: A Jedi's Return    3956
Length: 3957, dtype: int64

In [188]:
qualified.iloc[[0]]

,movie_id,title,release_date,popularity,vote_average,vote_count,overview,poster_path,genres,cast,crew,keywords,keywords_data,score
0,5,Four Rooms,1995-12-09,18.413,5.8,2380,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...","[{'adult': False, 'gender': 2, 'id': 3129, 'kn...","[{'adult': False, 'gender': 1, 'id': 3110, 'kn...","{'id': 5, 'keywords': [{'id': 612, 'name': 'ho...","[{'id': 612, 'name': 'hotel'}, {'id': 613, 'na...",5.948703


In [189]:
# 영화 제목을 입력 받으면 유사도가 높은 상위 10개 영화 목록 반환하는 함수

def get_recommendations(title, cosine_sim=cosine_sim):
    # 영화 제목에 해당하는 인덱스 번호
    idx = indices[title]    
    # 코사인 유사도 매트릭스에서 idx에 해당하는 데이터를 (idx, 유사도) 형식으로 반환
    sim_scores = list(enumerate(cosine_sim[idx]))
    # 코사인 유사도 기준 내림차순 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11] #자기자신 제외 10개 출력
    # 추천 영화 목록 10개 인덱스 정보 추출
    movie_indices = [i[0] for i in sim_scores]
    # 인덱스 정보로 영화 제목 출력
    return qualified['title'].iloc[movie_indices]

In [190]:
get_recommendations('Four Rooms')

2753                               Ted 2
3354                        Hotel Mumbai
3880                Friends: The Reunion
620     Bill & Ted's Excellent Adventure
257         There's Something About Mary
30                        Before Sunrise
118                         Hotel Rwanda
1581                         After Hours
811                                 1408
2659                Hotel Transylvania 2
Name: title, dtype: object

In [191]:
qualified['title'].head(20)

0                                            Four Rooms
1                                             Star Wars
2                                          Finding Nemo
3                                          Forrest Gump
4                                       American Beauty
5                                          Citizen Kane
6                                    Dancer in the Dark
7                                     The Fifth Element
8                                            Metropolis
9     Pirates of the Caribbean: The Curse of the Bla...
10                                    Kill Bill: Vol. 1
11                                              Jarhead
12                                       Apocalypse Now
13                                           Unforgiven
14                                   The Simpsons Movie
15                Eternal Sunshine of the Spotless Mind
16                                        Amores Perros
17           Pirates of the Caribbean: Dead Man'

In [192]:
get_recommendations('Finding Nemo')

2586                                         Finding Dory
2068                                           Mr. Nobody
1527                                           Shark Tale
573     Teenage Mutant Ninja Turtles II: The Secret of...
2619                                             Paperman
2874                                          Knock Knock
2218                          Don't Be Afraid of the Dark
2635                                           Cinderella
2556                                        Wolf Children
1464                                             Rounders
Name: title, dtype: object

In [193]:
# 감독 정보 추출
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [194]:
qualified['director'] = qualified['crew'].apply(get_director)
qualified['director']

C:\Users\yunyo\AppData\Local\Temp\ipykernel_13244\685247002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['director'] = qualified['crew'].apply(get_director)


0              Allison Anders
1                George Lucas
2              Andrew Stanton
3             Robert Zemeckis
4                  Sam Mendes
                ...          
3952    Matt Bettinelli-Olpin
3953             Marc Forster
3954              Ben Affleck
3955               Scott Mann
3956               David Gelb
Name: director, Length: 3957, dtype: object

In [196]:
# 출연 배우
qualified.loc[0,'cast']

[{'adult': False,
  'gender': 2,
  'id': 3129,
  'known_for_department': 'Acting',
  'name': 'Tim Roth',
  'original_name': 'Tim Roth',
  'popularity': 22.663,
  'profile_path': '/qSizF2i9gz6c6DbAC5RoIq8sVqX.jpg',
  'cast_id': 42,
  'character': 'Ted the Bellhop',
  'credit_id': '52fe420dc3a36847f80001b7',
  'order': 0},
 {'adult': False,
  'gender': 1,
  'id': 3130,
  'known_for_department': 'Acting',
  'name': 'Jennifer Beals',
  'original_name': 'Jennifer Beals',
  'popularity': 17.792,
  'profile_path': '/uJYKQC9GfRU3LHugCVDbAP1DrFH.jpg',
  'cast_id': 29,
  'character': 'Angela',
  'credit_id': '52fe420dc3a36847f8000183',
  'order': 1},
 {'adult': False,
  'gender': 2,
  'id': 3131,
  'known_for_department': 'Acting',
  'name': 'Antonio Banderas',
  'original_name': 'Antonio Banderas',
  'popularity': 26.423,
  'profile_path': '/n8YlGookYzgD3cmpMP45BYRNIoh.jpg',
  'cast_id': 31,
  'character': 'Man',
  'credit_id': '52fe420dc3a36847f800018b',
  'order': 2},
 {'adult': False,
  'gen

In [199]:
# 키워드
qualified.loc[0,'keywords_data']

[{'id': 612, 'name': 'hotel'},
 {'id': 613, 'name': "new year's eve"},
 {'id': 616, 'name': 'witch'},
 {'id': 622, 'name': 'bet'},
 {'id': 922, 'name': 'hotel room'},
 {'id': 2700, 'name': 'sperm'},
 {'id': 9706, 'name': 'anthology'},
 {'id': 12670, 'name': 'los angeles, california'},
 {'id': 160488, 'name': 'hoodlum'},
 {'id': 171366, 'name': 'multiple storylines'},
 {'id': 187056, 'name': 'woman director'}]

In [198]:
# 장르
qualified.loc[0, 'genres']

[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name': 'Comedy'}]

In [200]:
# 처음 5개 데이터 중 name에 해당하는 value만 추출
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 5:
            names = names[:5]
        return names
    return []

In [201]:
features = ['genres', 'cast', 'keywords_data']
for feature in features:
    qualified[feature] = qualified[feature].apply(get_list)

C:\Users\yunyo\AppData\Local\Temp\ipykernel_13244\3559133786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified[feature] = qualified[feature].apply(get_list)
C:\Users\yunyo\AppData\Local\Temp\ipykernel_13244\3559133786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified[feature] = qualified[feature].apply(get_list)
C:\Users\yunyo\AppData\Local\Temp\ipykernel_13244\3559133786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [202]:
qualified[['title', 'cast', 'director', 'keywords_data', 'genres']].head(3)

,title,cast,director,keywords_data,genres
0,Four Rooms,"[Tim Roth, Jennifer Beals, Antonio Banderas, V...",Allison Anders,"[hotel, new year's eve, witch, bet, hotel room]","[Crime, Comedy]"
1,Star Wars,"[Mark Hamill, Harrison Ford, Carrie Fisher, Pe...",George Lucas,"[android, galaxy, hermit, smuggling (contraban...","[Adventure, Action, Science Fiction]"
2,Finding Nemo,"[Albert Brooks, Ellen DeGeneres, Alexander Gou...",Andrew Stanton,"[parent child relationship, sydney, australia,...","[Animation, Family]"


In [203]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ', '')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ',''))
        else:
            return ''

In [204]:
features = ['director', 'cast', 'keywords_data','genres']
for feature in features:
    qualified[feature] = qualified[feature].apply(clean_data)

C:\Users\yunyo\AppData\Local\Temp\ipykernel_13244\2985393029.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified[feature] = qualified[feature].apply(clean_data)
C:\Users\yunyo\AppData\Local\Temp\ipykernel_13244\2985393029.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified[feature] = qualified[feature].apply(clean_data)
C:\Users\yunyo\AppData\Local\Temp\ipykernel_13244\2985393029.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [205]:
qualified[['title', 'cast', 'director', 'keywords_data', 'genres']].head(3)

,title,cast,director,keywords_data,genres
0,Four Rooms,"[timroth, jenniferbeals, antoniobanderas, vale...",allisonanders,"[hotel, newyear'seve, witch, bet, hotelroom]","[crime, comedy]"
1,Star Wars,"[markhamill, harrisonford, carriefisher, peter...",georgelucas,"[android, galaxy, hermit, smuggling(contraband...","[adventure, action, sciencefiction]"
2,Finding Nemo,"[albertbrooks, ellendegeneres, alexandergould,...",andrewstanton,"[parentchildrelationship, sydney,australia, ha...","[animation, family]"


In [206]:
def create_soup(x):
    return ' '.join(x['keywords_data']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
qualified['soup'] = qualified.apply(create_soup, axis=1)
qualified['soup']

C:\Users\yunyo\AppData\Local\Temp\ipykernel_13244\612666141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['soup'] = qualified.apply(create_soup, axis=1)


0       hotel newyear'seve witch bet hotelroom timroth...
1       android galaxy hermit smuggling(contraband) su...
2       parentchildrelationship sydney,australia harbo...
3       vietnamveteran posttraumaticstressdisorder hip...
4       adultery parentchildrelationship midlifecrisis...
                              ...                        
3952    newyorkcity mask trauma halloween college meli...
3953    basedonnovelorbook suicideattempt remake flash...
3954    sports basketball basedontruestory shoe during...
3955    sportclimbing gracecarolinecurrey virginiagard...
3956    behindthescenes makingof ewanmcgregor haydench...
Name: soup, Length: 3957, dtype: object

In [207]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(qualified['soup'])
count_matrix

<3957x13528 sparse matrix of type '<class 'numpy.int64'>'
	with 55079 stored elements in Compressed Sparse Row format>

In [208]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
cosine_sim2

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [209]:
indices

title
Four Rooms                            0
Star Wars                             1
Finding Nemo                          2
Forrest Gump                          3
American Beauty                       4
                                   ... 
Scream VI                          3952
A Man Called Otto                  3953
Air                                3954
Fall                               3955
Obi-Wan Kenobi: A Jedi's Return    3956
Length: 3957, dtype: int64

In [218]:
get_recommendations('Harry Potter and the Prisoner of Azkaban', cosine_sim2)

351              Harry Potter and the Goblet of Fire
393           Harry Potter and the Half-Blood Prince
349          Harry Potter and the Chamber of Secrets
1801    Harry Potter and the Deathly Hallows: Part 2
348         Harry Potter and the Philosopher's Stone
352        Harry Potter and the Order of the Phoenix
1800    Harry Potter and the Deathly Hallows: Part 1
903                             Howl's Moving Castle
2116                             Conan the Barbarian
2366                       Oz the Great and Powerful
Name: title, dtype: object

In [220]:
indices['Harry Potter and the Prisoner of Azkaban']

350

In [221]:
qualified.loc[350]

movie_id                                                       673
title                     Harry Potter and the Prisoner of Azkaban
release_date                                            2004-05-31
popularity                                                 110.979
vote_average                                                   8.0
vote_count                                                   19605
overview         Year three at Hogwarts means new fun and chall...
poster_path                       /aWxwnYoe8p2d2fcxOqtvAtJ72Rw.jpg
genres                                        [adventure, fantasy]
cast             [danielradcliffe, rupertgrint, emmawatson, rob...
crew             [{'adult': False, 'gender': 2, 'id': 491, 'kno...
keywords         {'id': 673, 'keywords': [{'id': 334, 'name': '...
keywords_data                 [flying, witch, bus, traitor, magic]
score                                                     7.936142
director                                             alfonsocu

In [222]:
qualified.loc[351]

movie_id                                                       674
title                          Harry Potter and the Goblet of Fire
release_date                                            2005-11-16
popularity                                                 141.916
vote_average                                                   7.8
vote_count                                                   18882
overview         When Harry Potter's name emerges from the Gobl...
poster_path                       /fECBtHlr0RB3foNHDiCBXeg9Bv9.jpg
genres                                [adventure, fantasy, family]
cast             [danielradcliffe, rupertgrint, emmawatson, bre...
crew             [{'adult': False, 'gender': 2, 'id': 293, 'kno...
keywords         {'id': 674, 'keywords': [{'id': 616, 'name': '...
keywords_data    [witch, basedonnovelorbook, worldcup, maze, ma...
score                                                     7.742004
director                                                mikene

In [223]:
movies = qualified[['movie_id', 'title', 'score', 'release_date', 'overview', 'genres', 'director', 'cast', 'keywords_data', 'poster_path']].copy()
movies.head()

,movie_id,title,score,release_date,overview,genres,director,cast,keywords_data,poster_path
0,5,Four Rooms,5.948703,1995-12-09,It's Ted the Bellhop's first night on the job....,"[crime, comedy]",allisonanders,"[timroth, jenniferbeals, antoniobanderas, vale...","[hotel, newyear'seve, witch, bet, hotelroom]",/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg
1,11,Star Wars,8.125107,1977-05-25,Princess Leia is captured and held hostage by ...,"[adventure, action, sciencefiction]",georgelucas,"[markhamill, harrisonford, carriefisher, peter...","[android, galaxy, hermit, smuggling(contraband...",/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg
2,12,Finding Nemo,7.738437,2003-05-30,"Nemo, an adventurous young clownfish, is unexp...","[animation, family]",andrewstanton,"[albertbrooks, ellendegeneres, alexandergould,...","[parentchildrelationship, sydney,australia, ha...",/eHuGQ10FUzK1mdOY69wF5pGgEf5.jpg
3,13,Forrest Gump,8.433292,1994-06-23,A man with a low IQ has accomplished great thi...,"[comedy, drama, romance]",robertzemeckis,"[tomhanks, robinwright, garysinise, sallyfield...","[vietnamveteran, posttraumaticstressdisorder, ...",/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg
4,14,American Beauty,7.889885,1999-09-15,"Lester Burnham, a depressed suburban father in...",[drama],sammendes,"[kevinspacey, annettebening, thorabirch, wesbe...","[adultery, parentchildrelationship, midlifecri...",/wby9315QzVKdW9BonAefg8jGTTb.jpg


In [ ]:
pickle.dump(movies, open('movies.pkl', 'wb'))
pickle.dump(cosine_sim2, open('cosine_sim2.pkl', 'wb'))